In [2]:
import polars as pl
from RecSys_Course_AT_PoliMi.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from pathlib import Path
from polimi.utils._custom import load_urms,load_sparse_csr
from polimi.utils._urm import build_recsys_features

2024-05-25 13:32:36.064220: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-25 13:32:38.115487: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load datasets


In [3]:
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_testset/articles.parquet')

history_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_large/train/history.parquet')

history_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_large/validation/history.parquet')

history_test = pl.read_parquet('/home/ubuntu/dataset/ebnerd_testset/test/history.parquet')
behaviors_test = pl.read_parquet('/home/ubuntu/dataset/ebnerd_testset/test/behaviors.parquet')


In [7]:
ICMs = load_urms(Path('/home/ubuntu/recsys_features/icm/testset/'))
ICMs

File loaded at: /home/ubuntu/recsys_features/icm/large/kenneth_title+subtitle.npz
File loaded at: /home/ubuntu/recsys_features/icm/large/title_embedding.npz
File loaded at: /home/ubuntu/recsys_features/icm/large/bert-base-multilingual-cased.npz
File loaded at: /home/ubuntu/recsys_features/icm/large/xlm-roberta-base.npz
File loaded at: /home/ubuntu/recsys_features/icm/large/document_vector.npz
File loaded at: /home/ubuntu/recsys_features/icm/large/emotion_scores.npz
File loaded at: /home/ubuntu/recsys_features/icm/large/contrastive_vector.npz


[<125541x384 sparse matrix of type '<class 'numpy.float64'>'
 	with 48207744 stored elements in Compressed Sparse Row format>,
 <125541x768 sparse matrix of type '<class 'numpy.float64'>'
 	with 96415488 stored elements in Compressed Sparse Row format>,
 <125541x768 sparse matrix of type '<class 'numpy.float32'>'
 	with 96415488 stored elements in Compressed Sparse Row format>,
 <125541x768 sparse matrix of type '<class 'numpy.float32'>'
 	with 96415488 stored elements in Compressed Sparse Row format>,
 <125541x300 sparse matrix of type '<class 'numpy.float32'>'
 	with 37662300 stored elements in Compressed Sparse Row format>,
 <125541x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 753246 stored elements in Compressed Sparse Row format>,
 <125541x768 sparse matrix of type '<class 'numpy.float32'>'
 	with 96415488 stored elements in Compressed Sparse Row format>]

In [51]:
URM_train = load_sparse_csr(Path('/home/ubuntu/recsys_features/urm/testset/URM_test.npz'))

recs = []

bert = ItemKNNCBFRecommender(URM_train=URM_train,ICM_train=ICMs[2])
bert.fit(similarity= 'euclidean', topK= 1457, shrink= 329, normalize_avg_row= True, similarity_from_distance_mode= 'exp', normalize= False) 
recs.append(bert)

contrastive = ItemKNNCBFRecommender(URM_train=URM_train,ICM_train=ICMs[6])
contrastive.fit(similarity= 'asymmetric', topK= 192, shrink= 569, asymmetric_alpha= 0.9094884938503743) 
recs.append(contrastive)

emotion = ItemKNNCBFRecommender(URM_train=URM_train,ICM_train=ICMs[5])
emotion.fit(similarity= 'euclidean', topK= 1099, shrink= 752, normalize_avg_row= True, similarity_from_distance_mode= 'lin', normalize= False) 
recs.append(emotion)

roberta = ItemKNNCBFRecommender(URM_train=URM_train,ICM_train=ICMs[3])
roberta.fit(similarity= 'cosine', topK= 363, shrink= 29) 
recs.append(roberta)

w_2_vec = ItemKNNCBFRecommender(URM_train=URM_train,ICM_train=ICMs[4])
w_2_vec.fit(similarity= 'cosine', topK= 359, shrink= 562) 
recs.append(w_2_vec)

kenneth = ItemKNNCBFRecommender(URM_train=URM_train,ICM_train=ICMs[0])
kenneth.fit(similarity= 'asymmetric', topK= 303, shrink= 574, asymmetric_alpha= 1.7852169782747023) 
recs.append(kenneth)

distilbert = ItemKNNCBFRecommender(URM_train=URM_train,ICM_train=ICMs[1])
distilbert.fit(similarity= 'asymmetric', topK= 921, shrink= 1, asymmetric_alpha= 0.774522157812755) 
recs.append(distilbert)



recsys_features = build_recsys_features(history=history_train.vstack(history_val).vstack(history_test),behaviors=behaviors_test,articles=articles,recs=recs)

File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_train.npz
ItemKNNCBFRecommender: URM Detected 3684 (19.6%) users with no interactions.
ItemKNNCBFRecommender: URM Detected 11952 (57.6%) items with no interactions.
Similarity column 20738 (100.0%), 133.09 column/sec. Elapsed time 2.60 min
ItemKNNCBFRecommender: URM Detected 3684 (19.6%) users with no interactions.
ItemKNNCBFRecommender: URM Detected 11952 (57.6%) items with no interactions.
Compute_Similarity: detected dense matrix
Similarity column 20738 (100.0%), 295.41 column/sec. Elapsed time 1.17 min
ItemKNNCBFRecommender: URM Detected 3684 (19.6%) users with no interactions.
ItemKNNCBFRecommender: URM Detected 11952 (57.6%) items with no interactions.
Similarity column 20738 (100.0%), 678.88 column/sec. Elapsed time 30.55 sec
ItemKNNCBFRecommender: URM Detected 3684 (19.6%) users with no interactions.
ItemKNNCBFRecommender: URM Detected 11952 (57.6%) items with no interactions.
Compute_Similarity: detected dense matrix

In [53]:
recsys_features

impression_id,article,user_id,recs0,recs1,recs2,recs3,recs4,recs5,recs6,recs7,recs8,recs9,recs10,recs11
u32,i32,u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
556008624,9193263,10068,0.00606,0.0,0.0,0.0,0.0,0.0,1.859136,0.0,0.0,0.00011,0.000007,0.0
556008624,9695098,10068,0.00303,0.0,0.0,0.0,0.0,0.0,0.964248,0.0,0.002663,0.009685,0.008015,0.003995
556008624,9758734,10068,0.00606,0.0,0.00133,0.0,0.0,0.0,1.951973,0.0,0.0,0.003549,0.000875,0.006811
385198819,9762352,10068,0.003029,0.0,0.0,0.923436,0.001626,0.0,1.946012,0.0,0.005326,0.02905,0.029936,0.006742
556008624,9772548,10068,0.00303,0.0,0.002659,0.923489,0.0,0.0,3.903791,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
567760513,9779705,2590637,0.018178,0.000884,0.00133,0.923348,0.0,0.000948,0.957195,0.0,0.0,0.0,0.0,0.0
567760515,9779705,2590637,0.018178,0.000884,0.00133,0.923348,0.0,0.000948,0.957195,0.0,0.0,0.0,0.0,0.0
567760515,9779723,2590637,0.021207,0.001255,0.003989,1.846684,0.001684,0.003742,1.887389,0.0,0.0,0.0,0.0,0.0


In [ ]:
couple = {
    'recs0': 'kenneth_emb_icm',
    'recs1': 'distilbert_emb_icm',
    'recs2': 'bert_emb_icm',
    'recs3': 'roberta_emb_icm',
    'recs4': 'distilbert_emb_icm',
    'recs5': 'emotions_emb_icm',
    'recs6': 'constrastive_emb_icm'
}


for col in recsys_features.columns:
    if couple[col] != None:
        recsys_features = recsys_features.rename({col: couple[col]})

In [ ]:
NORMALIZE_OVER_USER_ID = [
    'kenneth_emb_icm',
    'distilbert_emb_icm',
    'bert_emb_icm',
    'roberta_emb_icm',
    'distilbert_emb_icm',
    'emotions_emb_icm',
    'constrastive_emb_icm'
]
NORMALIZE_OVER_ARTICLE = [
    'kenneth_emb_icm',
    'distilbert_emb_icm',
    'bert_emb_icm',
    'roberta_emb_icm',
    'distilbert_emb_icm',
    'emotions_emb_icm',
    'constrastive_emb_icm'
]


recsys_features = recsys_features.with_columns(
    
    *[(pl.col(c) / pl.col(c).max().over(pl.col('user_id'))).alias(f'{c}_l_inf_user_id')
      for c in NORMALIZE_OVER_USER_ID],
    *[pl.col(c).std().over(pl.col('user_id')).alias(f'std_user_id_{c}')
      for c in NORMALIZE_OVER_USER_ID],
    *[pl.col(c).skew().over(pl.col('user_id')).alias(f'skew_user_id_{c}')
      for c in NORMALIZE_OVER_USER_ID],
    *[pl.col(c).kurtosis().over(pl.col('user_id')).alias(f'kurtosis_user_id_{c}')
      for c in NORMALIZE_OVER_USER_ID],
    *[pl.col(c).entropy().over(pl.col('user_id')).alias(f'entropy_user_id_{c}')
      for c in NORMALIZE_OVER_USER_ID],
    *[(pl.col(c) - pl.col(c).median().over(pl.col('user_id'))).alias(f'{c}_minus_median_user_id')
      for c in NORMALIZE_OVER_USER_ID],

    *[(pl.col(c) / pl.col(c).max().over(pl.col('article'))).alias(f'{c}_l_inf_article')
      for c in NORMALIZE_OVER_ARTICLE],
    *[pl.col(c).std().over(pl.col('article')).alias(f'std_article_{c}')
      for c in NORMALIZE_OVER_ARTICLE],
    *[pl.col(c).skew().over(pl.col('article')).alias(f'skew_article_{c}')
      for c in NORMALIZE_OVER_ARTICLE],
    *[pl.col(c).kurtosis().over(pl.col('article')).alias(f'kurtosis_article_{c}')
      for c in NORMALIZE_OVER_ARTICLE],
    *[pl.col(c).entropy().over(pl.col('article')).alias(f'entropy_article_{c}')
      for c in NORMALIZE_OVER_ARTICLE],
    *[(pl.col(c) - pl.col(c).median().over(pl.col('article'))).alias(f'{c}_minus_median_article')
      for c in NORMALIZE_OVER_ARTICLE],


)

In [ ]:
recsys_features

In [ ]:
recsys_features.write_parquet(Path('/home/ubuntu/recsys_features/testset.parquet'))